<a href="https://colab.research.google.com/github/HenriquesLab/rxiv-maker/blob/main/rxiv_maker_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RXiv-Maker: Automated LaTeX Article Generation

This notebook allows you to run RXiv-Maker in Google Colab to generate publication-ready LaTeX articles from Markdown manuscripts.

**Features:**
- Convert Markdown to LaTeX with academic formatting
- Generate figures from Python scripts and Mermaid diagrams
- Complete LaTeX compilation with bibliography
- Download generated PDF and source files

**What you'll need:**
- Your manuscript in Markdown format with YAML metadata
- Bibliography file (.bib)
- Figure source files (Python scripts or Mermaid diagrams)

## 🚀 Setup and Installation

First, let's install all the required dependencies.

In [ ]:
# Install LaTeX distribution
!apt-get update -qq
!apt-get install -y texlive-latex-extra texlive-fonts-recommended texlive-bibtex-extra biber

# Install Python dependencies
!pip install matplotlib>=3.7.0 seaborn>=0.12.0 numpy>=1.24.0 pandas>=2.0.0 scipy>=1.10.0
!pip install Pillow>=9.0.0 pypdf>=3.0.0 PyYAML>=6.0.0

# Install Mermaid CLI for diagram generation
!npm install -g @mermaid-js/mermaid-cli

print("✅ All dependencies installed successfully!")

## 📥 Clone RXiv-Maker Repository

Clone the RXiv-Maker repository to access the processing scripts and templates.

In [ ]:
import os
import shutil

# Clone the repository
if os.path.exists("rxiv-maker"):
    shutil.rmtree("rxiv-maker")

!git clone https://github.com/HenriquesLab/rxiv-maker.git
%cd rxiv-maker

# Create output directory
!mkdir -p output
!mkdir -p output/Figures

print("✅ RXiv-Maker repository cloned and set up!")
print(f"Current directory: {os.getcwd()}")
!ls -la

## 📝 Upload Your Files

Upload your manuscript and supporting files. You can either:
1. Use the sample files provided in the repository, or
2. Upload your own files using the file upload widget below

**Required files:**
- `00_ARTICLE.md` - Your manuscript with YAML frontmatter
- `02_REFERENCES.bib` - Your bibliography
- `FIGURES/` - Directory with figure source files (.py or .mmd)


In [ ]:
import os

from google.colab import files

print("Current files in the repository:")
!ls -la

print("\n" + "=" * 50)
print("OPTION 1: Use sample files (recommended for testing)")
print("The repository already contains sample files you can use.")
print("\nOPTION 2: Upload your own files")
print("Run the next cell to upload your manuscript and bibliography.")
print("=" * 50)

In [ ]:
# Uncomment and run this cell to upload your own files

# print("Upload your manuscript (00_ARTICLE.md):")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.md'):
#         os.rename(filename, '00_ARTICLE.md')
#         print(f"✅ Renamed {filename} to 00_ARTICLE.md")

# print("\nUpload your bibliography (02_REFERENCES.bib):")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith('.bib'):
#         os.rename(filename, '02_REFERENCES.bib')
#         print(f"✅ Renamed {filename} to 02_REFERENCES.bib")

# print("\nUpload figure source files (optional):")
# print("Upload .py or .mmd files that generate your figures")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     if filename.endswith(('.py', '.mmd')):
#         shutil.move(filename, f'FIGURES/{filename}')
#         print(f"✅ Moved {filename} to FIGURES/")

print("Using sample files from repository for demonstration.")
print("To use your own files, uncomment and run the code above.")

## 🔧 Generate Figures

Generate figures from Python scripts and Mermaid diagrams in the FIGURES directory.

In [ ]:
# Check what figure sources we have
print("Figure source files:")
!ls -la FIGURES/

# Generate figures
print("\n" + "=" * 50)
print("Generating figures...")
print("=" * 50)

!python3 src/py/commands/generate_figures.py --figures-dir FIGURES --output-dir FIGURES --format png

print("\n✅ Figure generation completed!")
print("Generated figures:")
!ls -la FIGURES/*.png 2>/dev/null || echo "No PNG files generated"
!ls -la FIGURES/*.pdf 2>/dev/null || echo "No PDF files generated"

## 📄 Generate LaTeX Article

Convert the Markdown manuscript to LaTeX using the RXiv-Maker processing pipeline.

In [ ]:
# Generate the LaTeX article
print("Generating LaTeX article from Markdown...")
print("=" * 50)

!python3 src/py/commands/generate_preprint.py --output-dir output

print("\n✅ LaTeX generation completed!")
print("\nContents of output directory:")
!ls -la output/

## 📚 Copy Supporting Files

Copy all necessary LaTeX style files, bibliography, and figures to the output directory.

In [ ]:
import glob
import shutil

print("Copying supporting files to output directory...")
print("=" * 50)

# Copy LaTeX style files
style_files = glob.glob("src/tex/style/*")
for file in style_files:
    if os.path.isfile(file):
        shutil.copy2(file, "output/")
        print(f"✅ Copied {os.path.basename(file)}")

# Copy bibliography
if os.path.exists("02_REFERENCES.bib"):
    shutil.copy2("02_REFERENCES.bib", "output/")
    print("✅ Copied bibliography")

# Copy figures
figure_files = glob.glob("FIGURES/*")
for file in figure_files:
    if os.path.isfile(file):
        shutil.copy2(file, "output/Figures/")
        print(f"✅ Copied figure {os.path.basename(file)}")

print("\n✅ All supporting files copied!")
print("\nFinal output directory contents:")
!ls -la output/
print("\nFigures directory:")
!ls -la output/Figures/

## 🔨 Compile LaTeX to PDF

Compile the LaTeX document to generate the final PDF. This process includes multiple passes for proper cross-references and bibliography.

In [ ]:
# Change to output directory for compilation
%cd output

print("Compiling LaTeX to PDF...")
print("This may take a few minutes...")
print("=" * 50)

# First LaTeX pass
print("🔄 First LaTeX pass...")
!pdflatex -interaction=nonstopmode ARTICLE.tex

# Bibliography processing
print("\n📚 Processing bibliography...")
!bibtex ARTICLE

# Second LaTeX pass
print("\n🔄 Second LaTeX pass...")
!pdflatex -interaction=nonstopmode ARTICLE.tex

# Third LaTeX pass (for final cross-references)
print("\n🔄 Final LaTeX pass...")
!pdflatex -interaction=nonstopmode ARTICLE.tex

print("\n" + "=" * 50)
if os.path.exists("ARTICLE.pdf"):
    print("🎉 PDF compilation completed successfully!")
    print(f"📄 Generated PDF: ARTICLE.pdf ({os.path.getsize('ARTICLE.pdf')} bytes)")
else:
    print("❌ PDF compilation failed. Check the log file for errors.")
    print("\nLaTeX log (last 50 lines):")
    !tail -50 ARTICLE.log

# Return to main directory
%cd ..

## 📱 Preview and Download

View the generated PDF and download all output files.

In [ ]:
import base64

from IPython.display import HTML, display

# Check if PDF was generated
pdf_path = "output/ARTICLE.pdf"
if os.path.exists(pdf_path):
    print("📄 PDF generated successfully!")

    # Display PDF preview
    with open(pdf_path, "rb") as f:
        pdf_data = f.read()

    pdf_b64 = base64.b64encode(pdf_data).decode("utf-8")
    pdf_display = f'<iframe src="data:application/pdf;base64,{pdf_b64}" width="100%" height="600px"></iframe>'
    display(HTML(pdf_display))

    print("\n" + "=" * 50)
    print("📥 Download your files:")
    print("1. Click the links below to download")
    print("2. Or run the next cell to download all files as ZIP")
    print("=" * 50)

else:
    print("❌ PDF not found. Please check the compilation step above.")
    print("\nAvailable files in output directory:")
    !ls -la output/

In [ ]:
# Download individual files
print("Click to download individual files:")
print("-" * 40)

# Download PDF
if os.path.exists("output/ARTICLE.pdf"):
    files.download("output/ARTICLE.pdf")
    print("✅ ARTICLE.pdf downloaded")

# Download LaTeX source
if os.path.exists("output/ARTICLE.tex"):
    files.download("output/ARTICLE.tex")
    print("✅ ARTICLE.tex downloaded")

print("\n📦 Or download everything as ZIP:")

In [ ]:
# Create ZIP archive with all output files
import datetime
import zipfile

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"rxiv_maker_output_{timestamp}.zip"

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    # Add all files from output directory
    for root, dirs, files_list in os.walk("output"):
        for file in files_list:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, "output")
            zipf.write(file_path, arcname)
            print(f"📁 Added to ZIP: {arcname}")

print(f"\n✅ Created ZIP archive: {zip_filename}")
print(f"📦 Archive size: {os.path.getsize(zip_filename)} bytes")

# Download the ZIP file
files.download(zip_filename)
print("🎉 ZIP archive downloaded successfully!")

## 🔧 Troubleshooting

If you encounter issues, run this cell to check common problems.

In [ ]:
print("🔍 TROUBLESHOOTING CHECKLIST")
print("=" * 50)

# Check required files
required_files = ["00_ARTICLE.md", "02_REFERENCES.bib"]
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file} found")
    else:
        print(f"❌ {file} missing")

# Check Python scripts
python_scripts = [
    "src/py/commands/generate_article.py",
    "src/py/commands/generate_figures.py",
]
for script in python_scripts:
    if os.path.exists(script):
        print(f"✅ {script} found")
    else:
        print(f"❌ {script} missing")

# Check LaTeX installation
import subprocess

try:
    result = subprocess.run(["pdflatex", "--version"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ pdflatex installed")
    else:
        print("❌ pdflatex not working")
except FileNotFoundError:
    print("❌ pdflatex not found")

# Check output directory
if os.path.exists("output"):
    print("✅ Output directory exists")
    files_in_output = os.listdir("output")
    print(f"📁 Files in output: {len(files_in_output)}")
    if "ARTICLE.tex" in files_in_output:
        print("✅ ARTICLE.tex generated")
    else:
        print("❌ ARTICLE.tex not found")
    if "ARTICLE.pdf" in files_in_output:
        print("✅ ARTICLE.pdf generated")
    else:
        print("❌ ARTICLE.pdf not found")
else:
    print("❌ Output directory missing")

print("\n" + "=" * 50)
print("If you see any ❌ errors above, please:")
print("1. Re-run the setup cells")
print("2. Check that your input files are properly formatted")
print("3. Look at the LaTeX log for compilation errors")

# Show recent LaTeX log if available
if os.path.exists("output/ARTICLE.log"):
    print("\n📄 Recent LaTeX log (last 20 lines):")
    !tail -20 output/ARTICLE.log

## 📖 Next Steps

**Congratulations!** You've successfully used RXiv-Maker to generate a publication-ready LaTeX article.

### What you can do next:

1. **Customize your manuscript**: Edit the YAML frontmatter in your Markdown file to change title, authors, affiliations, etc.

2. **Add more figures**: Create Python scripts or Mermaid diagrams in the FIGURES directory

3. **Modify the LaTeX template**: The template is in `src/tex/template.tex`

4. **Use locally**: Clone the repository to your local machine for offline use

### Learn more:
- [RXiv-Maker Documentation](https://github.com/HenriquesLab/rxiv-maker)
- [Markdown Guide](https://www.markdownguide.org/)
- [LaTeX Documentation](https://www.latex-project.org/help/documentation/)

### Need help?
- Open an issue on [GitHub](https://github.com/HenriquesLab/rxiv-maker/issues)
- Check the troubleshooting section above

---

*This notebook was created for the RXiv-Maker project by the HenriquesLab.*